## Problem Description
___
Determine a plan to minimize the cost of building bank branches in all counties where all counties with a branch must be adjacent to at least one county with principle place of business (PPB). Assume that none of the counties or any of their bordering counties currently has a PPB.

### Annualized Costs by county ($1,000's)

|County| Athens | Deleware | Fairfield | Fayette | Franklin | Gallia | Hocking | Jackson |  Knox | Licking |
|------|--------|----------|-----------|---------|----------|-------|---------|---------|-------|---------|  
| Cost | 1070   | 1420     | 1250      | 1300    | 1420     | 1310   | 1280    | 1140    | 1300  | 1310    |

|County| Logan | Madison | Meigs | Morrow | Pickaway | Pike | Ross | Union |  Vinton |
|------|-------|---------|-------|--------|----------|------|----- |-------|---------| 
| Cost | 1180  | 1310    | 1110  | 1180   | 1150     | 1110 | 1240 | 1230  | 1030    |



In [2]:
import numpy as np 
import pandas as pd
import pulp
import json

In [3]:
#fixed variables
counties = ['Athens','Delaware','Fairfield','Fayette', 'Franklin', 'Gallia', 'Hocking', 'Jackson', 'Knox', 'Licking', 'Logan', 'Madison',
            'Meigs', 'Morrow', 'Pickaway', 'Pike',	'Ross',	'Union', 'Vinton']

annualized_costs = [1070, 1420, 1250, 1300, 1420, 1310, 1280, 1140, 1300, 1310, 1180, 1310, 1110, 1180, 1150, 1110, 1240,
                    1230, 1030]

n_counties = len(counties)

file_name = 'county_adjacencies.csv'
adjacencies = pd.read_csv(file_name, index_col=[0])
adjacencies = adjacencies.fillna(0)

adjacencies.loc['Athens'].values

array([1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 1.])

In [10]:
#create the model
model = pulp.LpProblem(name='shift_planning', sense=pulp.LpMinimize)


county_variables = pulp.LpVariable.matrix('C', counties,
                                             lowBound=0, cat=pulp.LpBinary)

#Every county must have at least one adjacent county or itself with a PPB
for county in counties:
    model.addConstraint(pulp.LpConstraint(
        e=pulp.lpSum([adjacencies.loc[county][i]*county_variables[i] for i in range(n_counties)]) ,
        sense=pulp.LpConstraintGE,
        name='min_adj' + county,
        rhs=1))

objective = pulp.lpSum([county_variables[i]*annualized_costs[i] for i in range(n_counties)])

model.setObjective(objective)

model.solve()

if model.status == 1:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')
    print(f'objective: ${model.objective.value():,.0f}')
    output = []
    for i,var in enumerate(model.variables()):
        output.append(var.value())

    print(pd.DataFrame(np.array(output).reshape(1,19), columns=counties))
else:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')

status: 1, Optimal
objective: $5,770
   Athens  Delaware  Fairfield  Fayette  Franklin  Gallia  Hocking  Jackson  \
0     0.0       0.0        0.0      0.0       0.0     0.0      0.0      0.0   

   Knox  Licking  Logan  Madison  Meigs  Morrow  Pickaway  Pike  Ross  Union  \
0   1.0      0.0    1.0      0.0    0.0     0.0       1.0   1.0   0.0    0.0   

   Vinton  
0     1.0  
